# Orientações Gerais
* Link para API:  https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx
* É necessário que você tenha instalada a biblioteca **requests**

# Antes de cada célula é apresentada uma breve descrição do seu objetivo
* Carrega as bibliotecas necessárias para executar o código

In [13]:
import requests
from datetime import datetime
import excel as ex

* Seta a url base 
* Seta o ano que será utilizado como base para buscar os projetos propostos

In [14]:
url = 'https://splegisws.saopaulo.sp.leg.br/'
ano = '2023'

* Seta o endpoint que devolve todos os projetos de um determinado ano
* Seta o endpoint que devolve os autores dos projetos

In [15]:
projetos_por_ano = '/ws/ws2.asmx/ProjetosPorAnoJSON?Ano='+ano
projetos_autores = '/ws/ws2.asmx/ProjetosAutoresJSON'
r = requests.get(url+projetos_por_ano)
projetos_ano = r.json()

* Busca os dados dos endpoints citados anteriormente

**Obs:** Tempo da última execução: 1m 30s

In [16]:
TIPOS_PROJETOS = ('PDL','PL','PLO','PR')
rows = []

for projeto in projetos_ano:
    projeto_filter_dict = {}
    if projeto['tipo'] in TIPOS_PROJETOS:
        tipo_projeto = projeto['tipo']
        numero_projeto = projeto['numero']
        ano_projeto = projeto['ano']

        query = (f'{projetos_autores}?ano={ano_projeto}&tipo={tipo_projeto}&numero={numero_projeto}')
        autores_projeto = requests.get(url+query)
        autores_projeto_lista = autores_projeto.json()

        for data_autores in autores_projeto_lista:
            autores = data_autores['autores']
            autor = {}
            for autor in autores:
                autor['Projeto'] = tipo_projeto+str(numero_projeto)+"/"+str(ano_projeto)
                data_autores=datetime.strptime(projeto['data'][:10], "%Y-%m-%d")
                autor['Data'] = "{}/{}/{}".format(data_autores.day,data_autores.month,data_autores.year)
                autor['Nome Vereador'] = autor['nome']
                autor['Ementa'] = projeto['ementa']
                autor['Classificação'] = ''
                autor.pop('nome') 
                autor.pop('chave') 
                rows.append(autor)

* Utiliza os dados buscados na etapa anterior para construir uma planilha para cada vereador. As planilhas serão salvas na pasta projetos propostos

In [17]:
vereadores = {projeto['Nome Vereador'] for projeto in rows}

for vereador in vereadores:
    projeto_vereador = []
    for projeto in rows:
        if projeto['Nome Vereador'] == vereador:
            projeto_vereador.append(projeto)
    excel = ex.Excel(projeto_vereador,'projetos propostos')
    sheet_name = 'sheet_name' 
    table_name = vereador+'.xlsx' 
    excel.make_table(table_name, sheet_name)